In [1]:
import pandas as pd
import email
from email.parser import HeaderParser
from urlextract import URLExtract
import base64

master_urls = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/urlsinmails.csv"
master_headers = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/headersinmails.csv"
master_topdomains = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/topdomainsinmails.csv"
master_subdomains = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/subdomainsinmails.csv"
master_suffix = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/suffixinmails.csv"


urls_df = pd.read_csv(master_urls)
headers_df = pd.read_csv(master_headers)

#domain and subdomain must be a complex number

topdomains_df = pd.read_csv(master_topdomains)
subdomains_df = pd.read_csv(master_subdomains)
suffix_df = pd.read_csv(master_suffix)



In [2]:
meta_urls = {}
meta_headers = {}

#TODO: This method need lots of tuning
def get_email_as_dict(msg, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df ):
    
    #untuned code
    extractor = URLExtract()
    parser = email.parser.HeaderParser()
    headers = parser.parsestr(msg.as_string())

    metadata = {}
    all_urls = []
    all_headers = []
    
    for h in headers.items():
        urls = extractor.find_urls(h[1])
        all_urls = all_urls + urls
        #base64 decode
        if isBase64(h[1]):
            try:
                b64_h1= base64.b64decode(h[1]).decode("utf-8")
                urls = extractor.find_urls(str(b64_h1))
                all_urls = all_urls + urls

            except Exception as e:
                b64_h1 = ""
        # email_headers[h[0]] = [h[1]]
        all_headers.append(h[0])
        
    body = parse_body(msg)
    attchments = get_attachments(msg)
    str_body = str(body)

    urls = extractor.find_urls(str(body))
    
    all_urls = all_urls + urls
    all_urls_vectors = []
    all_topdomains_vectors = []
    all_subdomains_vectors = []
    all_suffix_vectors = []
    
    all_headers_vectors = []
    for each_url in all_urls:
        strip_url = each_url.lower().strip().replace("\\n","").replace("\\t","").replace("nhttp","http")
        sdomain,tdomain, suffix = get_domain(strip_url)
        #print("sdomain = ",sdomain,"tdomain = ",tdomain,"suffix = ",suffix, "url=",strip_url)

        try:
            url_index = urls_df["url"].to_list().index(strip_url) + 1
            all_urls_vectors.append(url_index)
        except ValueError as e:
            urls_df = pd.concat([urls_df,pd.DataFrame({"url":[strip_url]})], ignore_index = True)
            urls_df.reset_index()
            url_index = urls_df["url"].shape[0]
            all_urls_vectors.append(url_index)

        
        if sdomain != '':
            try:
                subdomain_index = subdomains_df["subdomain"].to_list().index(sdomain) + 1
                all_subdomains_vectors.append(subdomain_index)
            except ValueError as e:
                subdomains_df = pd.concat([subdomains_df,pd.DataFrame({"subdomain":[sdomain]})], ignore_index = True)
                subdomains_df.reset_index()
                subdomain_index = subdomains_df["subdomain"].shape[0]
                all_subdomains_vectors.append(subdomain_index)
        else:
            all_subdomains_vectors.append(0)

        try:
            topdomain_index = topdomains_df["topdomain"].to_list().index(tdomain) + 1
            all_topdomains_vectors.append(topdomain_index)
        except ValueError as e:
            topdomains_df = pd.concat([topdomains_df,pd.DataFrame({"topdomain":[tdomain]})], ignore_index = True)
            topdomains_df.reset_index()
            topdomain_index = topdomains_df["topdomain"].shape[0]
            all_topdomains_vectors.append(topdomain_index)
            
        if suffix != '':
            try:
                suffix_index = suffix_df["suffix"].to_list().index(suffix) + 1
                all_suffix_vectors.append(suffix_index)
            except ValueError as e:
                suffix_df = pd.concat([suffix_df,pd.DataFrame({"suffix":[suffix]})], ignore_index = True)
                suffix_df.reset_index()
                suffix_index = suffix_df["suffix"].shape[0]
                all_suffix_vectors.append(suffix_index)
        else:
            all_suffix_vectors.append(0)


            
    for each_header in all_headers:
        strip_header = each_header.lower().strip()
        try:
            header_index = headers_df["header"].to_list().index(strip_header) + 1
            all_headers_vectors.append(header_index)

        except ValueError as e:
            headers_df = pd.concat([headers_df,pd.DataFrame({"header":[strip_header]})], ignore_index = True)
            headers_df.reset_index()
            header_index = headers_df["header"].shape[0]
            all_headers_vectors.append(header_index)
        
    dict_row_urls = get_as_row(metatype="url",list_data=all_urls_vectors)
    dict_row_header = get_as_row(metatype="header",list_data=all_headers_vectors)
    dict_row_topdomain = get_as_row(metatype="topdomain",list_data=all_topdomains_vectors)
    dict_row_subdomain = get_as_row(metatype="subdomain",list_data=all_subdomains_vectors)
    dict_row_suffix = get_as_row(metatype="suffix",list_data=all_suffix_vectors)
    
    metadata.update(dict_row_urls) 
    metadata.update(dict_row_header)
    metadata.update(dict_row_topdomain)
    metadata.update(dict_row_subdomain)
    metadata.update(dict_row_suffix)
    metadata.update(attchments)
    
    return metadata, str_body,  urls_df,headers_df,topdomains_df,subdomains_df,suffix_df 



In [3]:
def get_email_as_df(msg, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df ):
    metadata, body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df  = get_email_as_dict(msg, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df )
    metadata_df = pd.DataFrame.from_dict([metadata])
    return metadata_df,body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df 
    

In [4]:
def isBase64(sb):
    try:
        if isinstance(sb, str):
                # If there's any unicode here, an exception will be thrown and the function will return false
                sb_bytes = bytes(sb, 'ascii')
        elif isinstance(sb, bytes):
                sb_bytes = sb
        else:
                raise ValueError("Argument must be string or bytes")
        return base64.b64encode(base64.b64decode(sb_bytes)) == sb_bytes
    except Exception:
                return False

In [5]:
import tldextract

def get_domain(urlwithdomain):
    res = tldextract.extract(urlwithdomain)
    if res[2] == '':
        return res[0], res[1] , res[2]
    else:
        return res[0], res[1] + "." + res[2], res[2]

In [6]:
def get_as_row(metatype,list_data):
    row = {}
    for index,eachitem in enumerate(list_data):
        row[metatype + str(index+1)] = eachitem
    return row

In [7]:
get_domain("http://google.com")

('', 'google.com', 'com')

In [8]:
# Reference: 
# https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not
#body = ""
def parse_body(email_msg):
    bodies = []  
    for part in email_msg.walk():
        ctype = part.get_content_type()
        cdispo = str(part.get('Content-Disposition'))
        if ctype == 'text/plain' and 'attachment' not in cdispo:
            bodies.append(part.get_payload(decode=True))
        else:
            bodies.append(email_msg.get_payload(decode=True))
        
    return bodies[0]

In [9]:
#Reference https://stackoverflow.com/questions/1936466/beautifulsoup-grab-visible-webpage-text

from bs4 import BeautifulSoup
from bs4.element import Comment

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [10]:
from scipy.stats import entropy
import pickle

def get_attachments(email_msg):
    attachments={}
    if email_msg == None:
        return {}
    attachment_count = 0
    for part in email_msg.walk():
        content_dis = part.get_content_disposition()
        if content_dis == "attachment":
            att_filename = part.get_filename()
            payload = bytearray(part.get_payload(decode=True))
            att_entropy= entropy(payload,base=2)
            att_size = len(payload)
            prefix = "attachment" + str(attachment_count)
            attachment = {}
            attachment[prefix + "_filename"] = att_filename
            attachment[prefix + "_entropy"] = att_entropy
            attachment[prefix + "_size"] = att_size
            #attachments.update(attachment)
            attachment_count+=1
    attachment_count = {}
    attachment_count["attachment_count"] = len(attachments)//3
    attachments.update(attachment_count)
    return attachments


In [15]:
import os
import traceback

data_path = "/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/phishingdata"
#data_path = "/Users/ravis/Downloads/apruvspam"
sub_folders = [x[0] for x in os.walk(data_path) if x[0] != data_path]
metadata_df = None
body_list = []
for folder in sub_folders:
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    for file in files:
        with open(f"{folder}/{file}", encoding="latin1") as f:
            f_realpath = os.path.realpath(f.name)
            is_spam = "spam" in f_realpath
            x = email.message_from_file(f)
            try:
                email_df, body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df  = get_email_as_df(x, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df )

                attachments = get_attachments(x)
                body_text = text_from_html(body[2:-1]).replace("\\n","").replace("\\t","")
                body_list.append([body_text, attachments['attachment_count'],1 if is_spam is True else 0])
                if metadata_df is None:
                    metadata_df = email_df
                else:
                    metadata_df = pd.concat([metadata_df,email_df], ignore_index=True)
            except Exception as e:
                print("--------------------------------starting------------")
                print(traceback.format_exc())
                print("--------------------------------ending------------")

                continue
                
body_df = pd.DataFrame(body_list, columns = ["body", "attachment_count","target"]) 
metadata_df["target"] = body_df["target"]
metadata_df["attachment_count"] = body_df["attachment_count"]


--------------------------------starting------------
Traceback (most recent call last):
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/3341638525.py", line 17, in <cell line: 9>
    email_df, body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df  = get_email_as_df(x, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df )
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/1963689943.py", line 2, in get_email_as_df
    metadata, body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df  = get_email_as_dict(msg, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df )
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/3848541455.py", line 17, in get_email_as_dict
    urls = extractor.find_urls(h[1])
  File "/Users/ravis/miniforge3/envs/tensorflow/lib/python3.9/site-packages/urlextract/urlextract_core.py", line 815, in find_urls
    url = next(urls, "")
  File "/Users/ravis/miniforge3/envs/tensorflow/li

/Users/ravis/miniforge3/envs/tensorflow/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.post-gazette.com/columnists/20020905brian5.asp\n\n\n\n" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/Users/ravis/miniforge3/envs/tensorflow/lib/python3.9/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.post-gazette.com/columnists/20020905brian5.asp\n\n\n\n" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


--------------------------------starting------------
Traceback (most recent call last):
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/3341638525.py", line 17, in <cell line: 9>
    email_df, body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df  = get_email_as_df(x, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df )
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/1963689943.py", line 2, in get_email_as_df
    metadata, body,urls_df,headers_df,topdomains_df,subdomains_df,suffix_df  = get_email_as_dict(msg, urls_df,headers_df,topdomains_df,subdomains_df,suffix_df )
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/3848541455.py", line 32, in get_email_as_dict
    attchments = get_attachments(msg)
  File "/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/2683923813.py", line 13, in get_attachments
    payload = bytearray(part.get_payload(decode=True))
TypeError: cannot convert 'NoneTy

/var/folders/rr/gnmws3gs2xn3p2l0t4_lq7hc0000gn/T/ipykernel_77245/3341638525.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  metadata_df["target"] = body_df["target"]


In [16]:
metadata_df = metadata_df.copy()

In [17]:
metadata_df

,attachment_count,url1,url2,url3,url4,url5,url6,url7,url8,url9,...,suffix246,suffix247,suffix248,suffix249,suffix250,suffix251,suffix252,suffix253,suffix254,target
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0,29213.0,7580.0,29214.0,29215.0,29216.0,29214.0,21061.0,7580.0,29214.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0,7580.0,29236.0,29237.0,29236.0,21061.0,7580.0,29236.0,29236.0,7580.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0,5600.0,29238.0,7580.0,29239.0,29240.0,29241.0,29241.0,29239.0,21061.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9964,0,20245.0,343.0,21874.0,343.0,23203.0,20603.0,1292.0,7063.0,23042.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9965,0,20245.0,343.0,21874.0,343.0,23481.0,4693.0,7063.0,20154.0,20245.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9966,0,20245.0,343.0,21874.0,7063.0,3244.0,23203.0,20603.0,1292.0,7063.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9967,0,20245.0,343.0,21874.0,343.0,7181.0,21505.0,1304.0,7063.0,6464.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
len(metadata_df.columns)

In [ ]:
body_df.to_csv("~/Downloads/bodytext.csv")


In [ ]:
mytext = "this contains ham"

is_spam = "spam" in mytext

print(1 if is_spam is True else 0)

In [ ]:
print(get_domain('http://hello.google.com'))

In [ ]:
urls_df

### Extract Domains from URLs

In [ ]:
urls_df = pd.read_csv(master_urls)


In [ ]:
subdomains = []
topdomains = []
suffixes = []
for index, row in urls_df.iterrows():
    url_data = row["url"]
    if url_data.startswith("http") == False:
        url_data = "http://" + url_data
    sub_domain, top_domain, suffix = get_domain(url_data)
    subdomains.append(sub_domain)
    topdomains.append(top_domain)
    
urls_df["subdomain"] = subdomains
urls_df["domain"] = topdomains
urls_df.to_csv("/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Code/extract_url_data/data/urls.csv")


In [ ]:
sub_df = pd.DataFrame(subdomains_uniq)
top_df = pd.DataFrame(topdomains_uniq)


In [ ]:
sub_df.to_csv("/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/subdomains.csv")
top_df.to_csv("/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/topdomains.csv")

In [18]:
metadata_df.to_csv("/Users/ravis/Library/CloudStorage/OneDrive-SouthernMethodistUniversity/CapstoneA/Data/FinalData/metadata.csv")



In [14]:
metadata_df

,url1,url2,url3,url4,url5,url6,url7,url8,url9,url10,...,header116,header117,header118,header119,header120,header121,header122,header123,header124,header125
0,343.0,23547.0,343.0,22463.0,22463.0,3454.0,7063.0,20818.0,20767.0,3453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,207.0,343.0,23547.0,343.0,23203.0,20603.0,1292.0,7063.0,22619.0,22619.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,343.0,23547.0,343.0,23203.0,20603.0,1292.0,7063.0,20640.0,7790.0,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,343.0,23547.0,343.0,23203.0,20603.0,1292.0,7063.0,23549.0,22823.0,5057.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,343.0,23547.0,343.0,23203.0,20603.0,1292.0,7063.0,21990.0,1489.0,23203.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,343.0,20046.0,343.0,23481.0,4693.0,7063.0,20154.0,20245.0,343.0,23481.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5619,20245.0,343.0,21874.0,343.0,23481.0,4693.0,7063.0,20154.0,20245.0,343.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5620,343.0,20046.0,343.0,7063.0,20245.0,343.0,28193.0,17517.0,17519.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5621,343.0,20046.0,343.0,6464.0,6464.0,1304.0,7063.0,6464.0,20245.0,343.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
